In [28]:
import pandas as pd

# Load your CSV data into a pandas DataFrame
df = pd.read_csv('data/crime/CrimeData-2023.csv')

In [29]:
# Violent and non-violent classification 
crime_categories = {
    # Violent Crimes
    'Assault Offenses': 'Violent',
    'Robbery': 'Violent',
    'Homicide Offenses': 'Violent',
    'Kidnapping/Abduction': 'Violent',
    'Animal Cruelty Offenses': 'Violent', # Adjust based on severity
    'Human Trafficking Offenses': 'Violent',
    'Sex Offenses': 'Violent',

    # Non-Violent Crimes
    'Larceny Offenses': 'Non-Violent',
    'Vandalism': 'Non-Violent',
    'Motor Vehicle Theft': 'Non-Violent',
    'Burglary': 'Non-Violent',
    'Fraud Offenses': 'Non-Violent',
    'Weapon Law Violations': 'Non-Violent', # May be violent in specific contexts
    'Drug/Narcotic Offenses': 'Non-Violent', # May be violent in specific contexts
    'Counterfeiting/Forgery': 'Non-Violent',
    'Prostitution Offenses': 'Non-Violent',
    'Stolen Property Offenses': 'Non-Violent',
    'Embezzlement': 'Non-Violent',
    'Extortion/Blackmail': 'Non-Violent',
    'Bribery': 'Non-Violent',

    # Grey Areas (Categorize based on specific data and context)
    'Pornography/Obscene Material': 'Grey Area',

}

# Map each crime to its category
df['CrimeCategory'] = df['OffenseCategory'].map(crime_categories)


In [30]:
# Convert 'OccurDate' to datetime
df['OccurDate'] = pd.to_datetime(df['OccurDate'])

# Extract month and year from 'OccurDate'
df['Month'] = df['OccurDate'].dt.month
df['Year'] = df['OccurDate'].dt.year
df['Occur_Month_Year'] = df['OccurDate'].dt.strftime('%Y-%m')

In [31]:
# Aggregate total counts by Neighborhood, Occur Month Year, and CrimeCategory
total_counts = df.pivot_table(index=['Neighborhood', 'Occur_Month_Year'],
                              columns='CrimeCategory',
                              values='OffenseCount',
                              aggfunc='sum',
                              fill_value=0).reset_index()

# Detailed aggregation by OffenseCategory
category_counts = df.pivot_table(index=['Neighborhood', 'Occur_Month_Year'],
                                 columns='OffenseCategory',
                                 values='OffenseCount',
                                 aggfunc='sum',
                                 fill_value=0).reset_index()


In [32]:
# Since we're working with pivot tables, direct merge is more straightforward
# Ensure both DataFrames have the same 'Neighborhood' and 'Occur_Month_Year' structure
final_summary = pd.merge(total_counts, category_counts, on=['Neighborhood', 'Occur_Month_Year'], how='outer')


In [33]:
final_summary.reset_index(inplace=True)

In [34]:
# Reset index if necessary (here it seems not needed as merge doesn't set an index)
print(final_summary.head())

   index Neighborhood Occur_Month_Year  Grey Area  Non-Violent  Violent  \
0      0      Alameda          2022-12          0            7        0   
1      1      Alameda          2023-01          0           17        0   
2      2      Alameda          2023-02          0           12        2   
3      3      Alameda          2023-03          0           13        0   
4      4      Alameda          2023-04          0           14        3   

   Animal Cruelty Offenses  Arson  Assault Offenses  Bribery  ...  \
0                        0      0                 0        0  ...   
1                        0      0                 0        0  ...   
2                        0      0                 2        0  ...   
3                        0      0                 0        0  ...   
4                        0      0                 3        0  ...   

   Kidnapping/Abduction  Larceny Offenses  Motor Vehicle Theft  \
0                     0                 4                    1   
1 

In [35]:
final_summary.to_csv('data/crime/aggregated_crime_data23v4.csv', index=False)